# import all the libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as rn
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from numpy import array
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LeakyReLU

from keras import backend as K
from keras.callbacks import History
from keras import optimizers
from keras.layers import Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard



# define paths to various directories


# change the version of proxies
version='v4'
keras_model_path=r"C:\Users\Keras-Model\\"

database_path=r"C:\Users\Database\\"
figure_path=r"C:\Users\Results\\"


#load scaler function and keras models

scalerX_fopr='scalerX_fopr_49'+version+'.gz'
scalerY_fopr='scalerY_fopr_49'+version+'.gz'
scalerX_fwpr="scalerX_fwpr_49"+version+'.gz'
scalerY_fwpr='scalerY_fwpr_49'+version+'.gz'
scalerX_fwir="scalerX_fwir_49"+version+'.gz'
scalerY_fwir='scalerY_fwir_49'+version+'.gz'


model_fopr=load_model(keras_model_path+"fopr_Olymp49-"+version+"_minmax.keras")
model_fwpr=load_model(keras_model_path+"fwpr_Olymp49-"+version+"_minmax.keras")
model_fwir=load_model(keras_model_path+"fwir_Olymp49-"+version+"_minmax.keras")   

import joblib
scalerX_fopr=joblib.load(keras_model_path+scalerX_fopr)
scalerY_fopr=joblib.load(keras_model_path+scalerY_fopr)
scalerX_fwpr=joblib.load(keras_model_path+scalerX_fwpr)
scalerY_fwpr=joblib.load(keras_model_path+scalerY_fwpr)
scalerX_fwir=joblib.load(keras_model_path+scalerX_fwir)
scalerY_fwir=joblib.load(keras_model_path+scalerY_fwir)
   
#NPV calculation
          
def NPV(Rate_Oil, Rate_Inj, Rate_Water, Oil_Price, Cost_wi, Cost_wp, disc):
#Oil price ($/bbl)60
#Cost produced water ($/bbl)6
#Cost injected water ($/bbl)2
#Annual discount factor
#oil rate in Scm3/day
#Cum_Oil is array of cummulative oil produced over the lifetime of reservoir
    NPV_calc=0
    for i in range(Rate_Oil.shape[0]):
        NPV_calc+= (Rate_Oil[i,0]*Oil_Price+Rate_Water[i,0]*Cost_wp+Rate_Inj[i,0]*Cost_wi)*365*6.29/(np.power((1+disc),i))
    return NPV_calc



def NPV_ecl(Rate_Oil, Rate_Inj, Rate_Water, Oil_Price, Cost_wi, Cost_wp, disc):
#Oil price ($/bbl)60
#Cost produced water ($/bbl)6
#Cost injected water ($/bbl)2
#Annual discount factor
#oil rate in Scm3/day
#Cum_Oil is array of cummulative oil produced over the lifetime of reservoir
    NPV_calc=0
    for i in range(Rate_Oil.shape[0]):
        NPV_calc+= (Rate_Oil[i]*Oil_Price+Rate_Water[i]*Cost_wp+Rate_Inj[i]*Cost_wi)*365*6.29/(np.power((1+disc),i))
    return NPV_calc

OSError: SavedModel file does not exist at: C:\Users\Keras-Model\\fopr_Olymp49-v4_minmax.keras/{saved_model.pbtxt|saved_model.pb}

## Calculate NPV_SPM and NPV_Eclipse for Test Data

In [ ]:
data = pd.read_csv(r"C:\Users\test.csv",sep=',')
X_test=data[["timestep", "BHPI1", "BHPI2", "BHPI3", "BHPI4", "BHPI5",\
            "BHPI6", "BHPP1", "BHPP10", "BHPP2", "BHPP3","BHPP4", "BHPP5",\
            "BHPP6", "BHPP7", "BHPP8", "BHPP9"]]


fopr_test=model_fopr.predict(scalerX_fopr.transform(X_test))
fopr_test=scalerY_fopr.inverse_transform(fopr_test)
fwpr_test=model_fwpr.predict(scalerX_fwpr.transform(X_test))
fwpr_test=scalerY_fwpr.inverse_transform(fwpr_test)
fwir_test=model_fwir.predict(scalerX_fwir.transform(X_test))
fwir_test=scalerY_fwir.inverse_transform(fwir_test)
NPV_SPM=NPV(fopr_test,fwir_test,fwpr_test,60,-2,-6,0.08)
print('NPV_SPM :', NPV_SPM)


best_fopr=data['FOPR']
best_fwpr=data['FWPR']
best_fwir=data['FWIR']
NPV_ECL=NPV_ecl(best_fopr,best_fwir,best_fwpr,60,-2,-4,0.08)
print("{:.6e}".format(NPV_ECL))
print('NPV_ECL :', NPV_ECL)

# Calculate NPV_Eclipse for entire Database

In [ ]:
data=pd.read_csv(database_path+'Dataset.csv')
data.head()
best_fopr=data['FOPR']
best_fwpr=data['FWPR']
best_fwir=data['FWIR']

NPV_Ecl=[]
n=0
Cost_wp=-6
Cost_wi=-2
Oil_Price=60
disc=0.08

for k in np.arange(150):
    
    Rate_Oil=best_fopr[n:n+20]
    Rate_Water=best_fwpr[n:n+20]
    Rate_Inj=best_fwir[n:n+20]
    NPV_calc=0
    #print(np.size(Rate_Inj))
    
    for i in range(20):
         NPV_calc+= (Rate_Oil[i+n]*Oil_Price+Rate_Water[i+n]*Cost_wp+Rate_Inj[i+n]*Cost_wi)*365*6.29/(np.power((1+disc),i))
    
            
    NPV_Ecl.append(NPV_calc)
    n=n+20
    
    np.savetxt('NPV_49ECL'+ version +'.csv', NPV_Ecl, delimiter = ",")
  

# Caluclate NPV_SPM for entire Dataset

In [ ]:

data = pd.read_csv(r"C:\Users\Database\Dataset.csv",sep=',')
X_data=data[["timestep", "BHPI1", "BHPI2", "BHPI3", "BHPI4", "BHPI5",\
            "BHPI6", "BHPP1", "BHPP10", "BHPP2", "BHPP3","BHPP4", "BHPP5",\
            "BHPP6", "BHPP7", "BHPP8", "BHPP9"]]

# Predict from Keras models
fopr=model_fopr.predict(scalerX_fopr.transform(X_data))
fopr=scalerY_fopr.inverse_transform(fopr).flatten()
fwpr=model_fwpr.predict(scalerX_fwpr.transform(X_data))
fwpr=scalerY_fwpr.inverse_transform(fwpr).flatten()
fwir=model_fwir.predict(scalerX_fwir.transform(X_data))
fwir=scalerY_fwir.inverse_transform(fwir).flatten()


NPV_SPM=[]
n=0
Cost_wp=-6
Cost_wi=-2
Oil_Price=60
disc=0.08
for k in np.arange(150):
    
    Rate_Oil=fopr[n:n+20]
    Rate_Water=fwpr[n:n+20]
    Rate_Inj=fwir[n:n+20]
    NPV_calc=0
    #print(Rate_Oil)
    #print(np.size(Rate_Water))
    for i in np.arange(20):
              
        NPV_calc+= (Rate_Oil[i]*Oil_Price+Rate_Water[i]*Cost_wp+Rate_Inj[i]*Cost_wi)*365*6.29/(np.power((1+disc),i))
    
    
            
    NPV_SPM.append(NPV_calc)
    n=n+20
    np.savetxt("NPV_49SPM"+version+".csv",NPV_SPM, delimiter = ",")
    